In [1]:
import cv2

from matplotlib import pyplot as plt
import numpy as np
import time
import os

import pygacal

from pygacal.common.cgatools import *

#   from pygacal import geometry, camera, rotation

from pygacal.geometry.transformations import *

from pygacal.rotation import minimizeError
from pygacal.rotation.mapping import BivectorLineImageMapping
#from clifford_tools.numerical.applications.vision.core import Camera 

from pygacal.camera.projection import projectLineToPlane
from pygacal.geometry.lines import extract2Dparams, createLine

from os.path import join, abspath

from resources.squash_court.court_image import *
from resources.squash_court.court_model import *

def calibrate(model, image_model):
    lines_ga = model.get_lines_ga()
    lines_img_ga = img_model.get_lines_img_ga()


    t0 = time.clock()

    R_min, Nint = minimizeError((lines_ga, lines_img_ga), 
                                BivectorLineImageMapping, 
                                x0 = None)
    print("t = ", time.clock()- t0, "N_iter = ", Nint)

    return R_min


def project_lines_matrix(model, image_model, P):
    def image_projection(x, R_mat, K):
        X = np.array(np.dot(K, np.dot(R_mat, x)[:3])).reshape(-1)
        return X[:2]/X[2]

    lines_ga = model.get_lines_ga()
    lines_img_ga = img_model.get_lines_img_ga()

    K = img_model.get_internal_calibration()

    image_name = img_model.get_image_name()
    img        = cv2.imread(image_name, cv2.IMREAD_COLOR)


    for line in model.get_lines():

        start = np.int64(image_projection(line[0], P, K))
        end = np.int64(image_projection(line[1], P, K))
        cv2.line(img, tuple(start), tuple(end), (255, 0, 0), 1)
    
    for line in image_model.get_lines():
        start = line[0][:2]
        end = line[1][:2]
        cv2.line(img, tuple(start), tuple(end), (0, 255, 0), 1)

    cv2.imshow("Projection", img)
    cv2.waitKey(0)


def project_lines(model, image_model, R):
    
    lines_ga = model.get_lines_ga()
    lines_img_ga = img_model.get_lines_img_ga()

    K = img_model.get_internal_calibration()

    image_name = img_model.get_image_name()
    img        = cv2.imread(image_name, cv2.IMREAD_COLOR)


    def to_homogenous_coords(vec2):
        ans = np.ones(3)
        ans[:2] = vec2
        return ans

    for line in lines_ga:
        size = 1 #Some scaling parameter 

        line_img_projected = projectLineToPlane(line, R)
                
        a, ma = extract2Dparams(line_img_projected)
        
        A = a + size*ma
        B = a - size*ma
                
        
        #Why is numpy so broken!! 
        A = np.reshape(np.array(np.dot(K, to_homogenous_coords(A))), (3, ))
        B = np.reshape(np.array(np.dot(K, to_homogenous_coords(B))), (3, ))

        start = tuple(np.int32(A[:2]))
        end   = tuple(np.int32(B[:2]))

        #Blue from model
        cv2.line(img, start, end, (255, 0, 0), 1)

    for line in lines_img_ga:
        size = 1 #Some scaling parameter 

                
        a, ma = extract2Dparams(line)
        
        A = a + size*ma
        B = a - size*ma
                        
        #Why is numpy so broken!! 
        A = np.reshape(np.array(np.dot(K, to_homogenous_coords(A))), (3, ))
        B = np.reshape(np.array(np.dot(K, to_homogenous_coords(B))), (3, ))

        start = tuple(np.int32(A[:2]))
        end   = tuple(np.int32(B[:2]))

        #Green from image
        cv2.line(img, start, end, (0, 255, 0), 1)

        
    cv2.imshow("Projection", img)
    cv2.waitKey(0)


ImportError: No module named 'common'

In [ ]:
img_model = RobertSquashCourtImage()
model     = SquashCourtModel()

R_min = calibrate(model, img_model)
project_lines(model, img_model, R_min)
theta, t = versor_to_param(R_min)

print(versor_to_param(R_min))

P = np.linalg.inv(full_projection_matrix(theta, t))

project_lines_matrix(model, img_model, P)
